<h1><center>Final Assignement</center></h1>

Student number: 200420653

<a class="anchor" id="0"></a>
## Table of content
1. [Introduction and objectives](#1)<br>
    1.1 [Introduction](#1.1)<br>

2. [Data](#2)<br>
    2.1 [Data sources](#2.1)<br>
    2.2 [Data cleaning](#2.2)<br>
    2.3 [Data preparation](#2.3)<br>

4. [Summary](#4)<br>

**Doings:** 
<br>

Code
### - zeigen, dass Vorgehensweise dem 7 Schritte Ansatz von Chollet entspricht
- check if after first transformations a reset_index(true) is needed
- plotting the data to get some insights
- show picture of formula for RMSSE from directory ../res/misc/Formula.png
- Feature Selection: Reduce the dimensionality of data by selecting only the most important features - may involve domain knowledge or techniques like PCA or feature importance scoring.
- merge df with weekdays
- only read in data in dataframes that is needed. In case something not needed, delete it
-create classes; use special skills like the super class or an inheritance from another class; ask gpt which special skills could make sense
- If I reduce the size of the df and hash the values to integers the question remains if the models predictions would have been better without that conversion + check if float32 conversion decreases accuracy of models
- Use scalene to measure the performance of the code
- clarify how the 7 step approach from chollet book is used
- dataframe: hash id's to decrease size of df drastically. Could speed up all computations
- maybe normalization makes sense but beware of fact that only reading operations are done on the data
- it seems like I have use tensorflow, so keep in mind that it is used somewhere. Maybe just keras works

Report
- correct citation style with ACM and Mendeley

Information
- sales_amount values are originally d_1, d_2 and so on and either int16 or int8, so should not be a float value

In [ ]:
#Import data handling libraries
import pandas as pd
import numpy as np

In [ ]:
#Specify directories

###local###
#get parent folder of current directory
parent_dir = '/Users/mf/Desktop/CS/Studies/7_Final_Project/Kaggle_M5PointPrediction'

#Directory resources
res_dir = parent_dir + '/res/'
src_dir = parent_dir + '/src/'

###On Kaggle###
#res_dir = '/kaggle/input/m5-forecasting-accuracy/'
# ?? src_dir = '/kaggle/input/m5-forecasting-accuracy/src/'


print('Parent directory of current notebook: ' + parent_dir)
print('Resource directory: ' + res_dir)

In [ ]:
# Import the provided csv files
df_cal = pd.read_csv(res_dir + 'calendar.csv')
df_prices = pd.read_csv(res_dir + 'sell_prices.csv')
df_train_eval = pd.read_csv(res_dir + 'sales_train_evaluation.csv')
#df_train_val = pd.read_csv(res_dir + 'sales_train_validation.csv')
#df_sample_subm = pd.read_csv(res_dir + 'sample_submission.csv')

In [ ]:
#Inspect dataframes
# print('\nCalendar dataframe: ')
# print(df_cal.head())
# print('\nPrices dataframe: ')
# print(df_prices.head())
# print('\nTrain evaluation dataframe: ')
# print(df_train_eval.head())


In [ ]:
#Helper function to reduce memory usage of dataframes
def reduce_df_mem_usage(df, df_name):
    """ 
    Helper function to iterate all columns of given dataframe and check and set for smallest dtype to reduce memory usage
    Taken and adapted from the widely used function which is available for instance here: 
    https://www.kaggle.com/code/gemartin/load-data-reduce-memory-usage/notebook
    There is a flaw in the integer section in the publicly available function which is that it possibly introduces rounding errors when converting to a smaller dtype
    """

    #Print original memory usage
    print('Dataframe ' + df_name + ' is being processed...')
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    #Iterate through each column
    for col in df.columns:
        #Get dtype
        col_type = df[col].dtype
        
        #try except throw block to try if the following code works, otherwise skip this loop
        try:

            #If type is not an object, therefore numerical, get biggest and smallest values
            if col_type != object:
                c_min = df[col].min()
                c_max = df[col].max()
                
                #If type is int
                if str(col_type)[:3] == 'int':
                    #If min value is greater than min value of given dtype and max value is smaller than max value of given dtype -> adjust dtype
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)
                #Downcasting of float values leads to rounding errors -> as precision is crucial for float values, no downcasting is performed

            #If none of the above, then assumption that  finite set of possible values -> convert to category which is internally stored as int but when queried returns the string
            else:
                df[col] = df[col].astype('category')
        
        except:
            pass

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    print('---------------------------------------------------\n')

    return df


# Reduce size for float when precision is not really needed
            # else:
                # if str(col_type)[:7] == 'float64':
                #     #calculate difference between original column and downcasted column
                #     diff_64_32 = np.abs(df[col] - df[col].astype(np.float32)).round(2)
                #     # If the max difference is below the threshold, downcast column
                #     if diff_64_32.max() < prec_threshold:
                #         diff_32_16 = np.abs(df[col] - df[col].astype(np.float16)).round(2)
                #         if diff_32_16.max() < prec_threshold:
                #             df[col] = df[col].astype(np.float16).round(2)
                #         else: df[col] = df[col].astype(np.float32).round(2)

## Memory Reducer
# :df pandas dataframe to reduce size             # type: pd.DataFrame()
# :verbose                                        # type: bool
def reduce_df_mem_usage(df, df_name, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                       df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
#Read in all data used later on
df_prices = reduce_df_mem_usage(df_prices, 'df_prices')
df_cal = reduce_df_mem_usage(df_cal, 'df_cal')
df_train_eval = reduce_df_mem_usage(df_train_eval, 'df_train_eval')

#delete dataframe from memory or just don't read in
#del df_sample_subm
#get name of dataframe


In the following I will create 3 dataframes that will be stored in pickle format so that unused dataframes can be deleted from memory and the information is clearly separated.
The dataframes are:
- 1. df_conv: base grid with the main train information that contains the conversion of the sales data from wide to long format and merge of the sales price
- 2. calendar: contains calendar data and some generated date features
- 3. XXX: Feature engineering

First convert from wide to long format of the training dataframe. Right now all sales days are in columns. We want them in rows for processing in ML / DL models.

In [ ]:
# countrows df_train_eval
#df_train_eval.shape
#df_train_eval.head()
#df_cal.head()
#df_prices.head()

In [ ]:
# Convert from wide to long format
df_conv = pd.melt(df_train_eval,
                    id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], 
                    var_name='d', 
                    value_name='sales_amount')

# delete df_train_eval from memory because no longer needed
del df_train_eval 

# Reduce memory usage of df_conv
df_conv = reduce_df_mem_usage(df_conv, 'df_conv')

Now we have the issue, that 0 values in the sales_amount column could mean that there were no sales, it could also mean that the sales start of the product was later and the product was just not available.
The sales start date can be infered from the price df. If there is no sales week for a product then that most likely means it wasn't available for sale.
I will first try to delete all rows from df_conv where there were no weeks available for from the price df. This will be done by adding the week information from prices through a left join. Then the week information is available to make a right join on the prices table and thereby filter out all rows where there was no sales week available (the product didn't exist yet).

The reason why I didn't convert float types is that they loose precision. That is absolutely crucial for sales prediction. That is why now the dataframe is too large for the merge operation in the following. Even 30GB RAM machines run out of memory on this computation as it require much more memory than the original ~680MB for df_conv and ~85MB for Prices. The solution for me is using DASK which handles operations on dataframes in smaller chunks.

In [ ]:
####### Cited from the 1st solution (exact citing) #######
## Merging by concat to not lose dtypes
def merge_by_concat_left(df1, df2, merge_on):
    merged_gf = df1[merge_on]
    merged_gf = merged_gf.merge(df2, on=merge_on, how='left')
    new_columns = [col for col in list(merged_gf) if col not in merge_on]
    df1 = pd.concat([df1, merged_gf[new_columns]], axis=1)
    return df1

In [ ]:
# Combine the converted df with the calendar df
df_conv = merge_by_concat_left(df_conv, df_cal[['d', 'wm_yr_wk']], ['d'])

In [ ]:
# Check output of operation
df_conv.head()

Merge with the prices df to get the sales start date for each product. Then filter out all rows where there was no price available (the product didn't exist yet).

In [ ]:
# Feature engineering for prices can be done here, and then can in the next step be merged with the df_conv dataframe and deleted from memory
# ...



# Combine the converted df with the prices df
df_conv = merge_by_concat_left(df_conv, df_prices, ['store_id', 'item_id', 'wm_yr_wk'])

In [ ]:
# Create a new column is_available where when column sell_price is NaN the value is 0 (indicates item was not available) and otherwiese 1 (indicates the item was available that day)
df_conv['is_available'] = np.where(df_conv['sell_price'].isna(), 0, 1).astype(np.int8)

# Also set column sales_amount to NaN when sell_price is NaN
df_conv['sales_amount'] = np.where(df_conv['sell_price'].isna(), np.nan, df_conv['sales_amount'])

In [ ]:
# Check output of operation
df_conv.head()

In [ ]:
# delete df_prices from memory because no longer needed
del df_prices
del df_conv['wm_yr_wk']
df_conv.reset_index(drop=True, inplace=True)

In [ ]:
# Muss ich folgenden Schritt aus Koreanischer Lösung nehmen? -->Comment MF: create a new empty dataframe which gets appended to the original one for the next 28 days (prediction period) and fill with NaNs

In [ ]:
# Base steps are done, now lets save to pickle file
df_conv.to_pickle(src_dir + 'processed_data/' + 'df_1.pkl')
del df_conv

Before we move on to the merge the infomation from the calendar to the df_conv dataframe, it makes sense to perform some feature engineering based on the prices information as long as the df_conv is smaller without the calendar information.

In [ ]:
# <<< Perform feature engineering based on prices on the df_conv dataframe as was done in the leading solution >>>

### Create the second dataframe with the calendar information
Now lets get the calendar information and do some feature engineering for a third dataframe.

In [ ]:
# read df_conv from pickle file 
df_conv = pd.read_pickle(src_dir + 'processed_data/' + 'df_1.pkl')

In [ ]:
# Retrieve only identifying columns of df_conv to keep df small and to merge with df_conv
df_conv = df_conv[['id', 'd']]

In [ ]:
# Pick all useful columns for merge with df_conv
icols = ['date',
         'd',
         'wday',
         'event_name_1',
         'event_type_1',
         'event_name_2',
         'event_type_2',
         'snap_CA',
         'snap_TX',
         'snap_WI']

# Now merge the df_conv with useful features from the prices df
df_conv = merge_by_concat_left(df_conv, df_cal[icols], ['d'])

In [ ]:
# Convert the date column to datetime format
df_conv['date'] = pd.to_datetime(df_conv['date'])

# Do feature engineering on the df_conv dataframe; more features are possible
df_conv['mday'] = df_conv['date'].dt.day.astype(np.int8)
df_conv['week'] = df_conv['date'].dt.isocalendar().week.astype(np.int8)
df_conv['month'] = df_conv['date'].dt.month.astype(np.int8)
df_conv['year'] = df_conv['date'].dt.year.astype(np.int16)
# df_conv['tm_y'] = (df_conv['tm_y'] - df_conv['tm_y'].min()).astype(np.int8)
# df_conv['tm_wm'] = df_conv['tm_d'].apply(lambda x: ceil(x/7)).astype(np.int8)

# df_conv['tm_dw'] = df_conv['date'].dt.dayofweek.astype(np.int8) 
# df_conv['tm_w_end'] = (df_conv['tm_dw']>=5).astype(np.int8)

df_conv['d'] = df_conv['d'].str.replace('d_', '').astype(np.int16)

# Date not needed anymore for final training df
del df_conv['date']
del df_conv['id']

In [ ]:
# save to pickle and delete df_cal from memory because no longer needed
df_conv.to_pickle(src_dir + 'processed_data/' + 'df_2.pkl')
del df_cal
del df_conv

# Now the date column with the old format can be deleted from the df_conv dataframe
df_conv = pd.read_pickle(src_dir + 'processed_data/' + 'df_1.pkl')
del df_conv['d']
df_conv.to_pickle(src_dir + 'processed_data/' + 'df_1.pkl')